# Reproducing results from the Multi-News paper

In this notebook we will try to reproduce the results from  the [Multi-News](https://www.aclweb.org/anthology/P19-1102/) [(GitHub)](https://github.com/Alex-Fabbri/Multi-News). 


They use the [OpenNMT](https://opennmt.net/) neural machine translation system, but adapted it some places.

The main steps of a OpenNMT-pipeline are
- Preprocessing 
- Model training
- Translate

You can find the documentation [here](https://opennmt.net/OpenNMT-py/quickstart.html#step-1-preprocess-the-data).


See also the python implementation of OpenNMT [here](https://github.com/OpenNMT/OpenNMT-py).

## Preparing Data

The data consists of parallel source (src) and target (tgt) data containing one sentence per line with tokens separated by a space.
We won't need to do preprocessing on our own, since preprocessed data is already provided

In [1]:
!dir

code  data  output  README.md  Train_BERT_HiPMAP.ipynb	Train_HiPMAP.ipynb


In [1]:
!pip install torchtext nltk opencv-python transformers==3.1.0

In [3]:
!python code/HiPMAP/preprocess.py \
    -train_src ../news-opinion-summarization/data/multi_news/raw/train.src.cleaned \
    -train_tgt ../news-opinion-summarization/data/multi_news/raw/train.tgt \
    -valid_src ../news-opinion-summarization/data/multi_news/raw/val.src.cleaned \
    -valid_tgt ../news-opinion-summarization/data/multi_news/raw/val.tgt \
    -save_data ../news-opinion-summarization/data/multi_news/newser_sent_500/newser_sents \
    -src_seq_length 10000 \
    -tgt_seq_length 10000 \
    -src_seq_length_trunc 500 \
    -tgt_seq_length_trunc 300 \
    -dynamic_dict \
    -share_vocab \
    -max_shard_size 10000000


Please backup existing pt file: ../news-opinion-summarization/data/multi_news/newser_sent_500/newser_sents.train*.pt, to avoid tampering!


## Train the Model



In [8]:
%%time
!CUDA_VISIBLE_DEVICES=0,1 python code/Hi_MAP/train.py \
    -save_model output/summarisation/model_newser_himap/Okt17_ \
    -data ../news-opinion-summarization/data/multi_news/final_preprocessed/final \
    -copy_attn -accum_count 5\
    -global_attention mlp \
    -word_vec_size 128 \
    -rnn_size 512  -layers 1 \
    -encoder_type brnn \
    -train_steps 20000 \
    -max_grad_norm 4 \
    -dropout 0. \
    -batch_size 2 \
    -optim adagrad \
    -learning_rate 0.15 \
    -adagrad_accumulator_init 0.1 \
    -reuse_copy_attn \
    -copy_loss_by_seqlength \
    -bridge \
    -seed 777 \
    -world_size 1  \
    -gpu_ranks 0 \
    -save_checkpoint_steps 1000 \
    -model_type fp16 \
    -train_from output/summarisation/model_newser_himap/Okt17__step_9000.pt

[2020-10-19 07:31:00,016 INFO] Loading checkpoint from output/summarisation/model_newser_himap/Okt17__step_9000.pt
[2020-10-19 07:31:02,966 INFO] Loading vocab from checkpoint at output/summarisation/model_newser_himap/Okt17__step_9000.pt.
/opt/conda/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
[2020-10-19 07:31:02,993 INFO]  * vocabulary size. source = 50004; target = 50004
[2020-10-19 07:31:02,993 INFO] Building model...
[2020-10-19 07:31:05,035 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50004, 128, padding_idx=1